### Split the data into train set and test set

In [ ]:
from src import DAPSAudioDataset
import os
import librosa
from torch.utils.data import DataLoader, Subset

import torch

audio_dir = "daps"

class_1_speakers = ["f1", "f7", "f8", "m3", "m6", "m8"]

class_mapping = {}

for root, dirs, files in os.walk('precomputed_spectrograms_aug_2'):
    for file in files:
        audio_path = os.path.join(root, file)
        try:
            speaker_prefix = file.split("_")[0]
            if speaker_prefix in class_1_speakers:
                class_mapping[audio_path] = 1
            else:
                class_mapping[audio_path] = 0
        except Exception as e:
            print(f"Error processing {audio_path}: {e}")

print("Class mapping created:")

class_0_count = sum(1 for label in class_mapping.values() if label == 0)
class_1_count = sum(1 for label in class_mapping.values() if label == 1)
print(f"Total Class 0 samples: {class_0_count}")
print(f"Total Class 1 samples: {class_1_count}")
print("Preparing the dataset...")

full_dataset = DAPSAudioDataset(class_mapping=class_mapping)

test_ids =[]
train_ids =[]
filenames = set()
f = open("demofile2.txt", "a")
for idx in range(len(full_dataset)):
    path, label = full_dataset.class_mapping[idx]
    if path is not None:
        filename = os.path.basename(path)
            
        filename_without_ext = os.path.splitext(filename)[0]
        parts = filename_without_ext.split('_')
            
        group = parts[0]
        place = ""
        if 'script1' in filename_without_ext:
            if 'aug' not in filename_without_ext:
                filenames.add(place)
                test_ids.append(idx)
            else: 
                continue
        else:
            train_ids.append(idx) 
            f.write(f"{filename_without_ext}\n")
print(len(test_ids))
print(len(train_ids))
f.close()

print("Dataset created")

train_dataset = Subset(full_dataset, train_ids)
test_dataset = Subset(full_dataset, test_ids)

def collate_fn(batch):
    batch = [item for item in batch if item is not None]
    if len(batch) == 0:
        return None
    return torch.utils.data.dataloader.default_collate(batch)


train_loader = DataLoader(
    train_dataset, batch_size=64, shuffle=True, collate_fn=collate_fn
)

test_loader = DataLoader(
    test_dataset, batch_size=16, shuffle=True, collate_fn=collate_fn
)

for batch in train_loader:
    inputs, labels = batch
    print(f"Train Inputs shape: {inputs.shape}")
    print(f"Train Labels: {labels}")
    break

for batch in test_loader:
    inputs, labels = batch
    print(f"Test Inputs shape: {inputs.shape}")
    print(f"Test Labels: {labels}")
    break

Class mapping created:
Total Class 0 samples: 1050
Total Class 1 samples: 900
Preparing the dataset...
300
1560
Dataset created


d:\WUT\ML\MLProject\src\pytorch_datasets.py:24: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  spectrogram_tensor = torch.load(spectrogram_path)


Train Inputs shape: torch.Size([64, 1, 64, 64])
Train Labels: tensor([0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
        1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
        0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1])
Test Inputs shape: torch.Size([16, 1, 64, 64])
Test Labels: tensor([0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1])


In [13]:
print(len(train_loader))

98


In [14]:
len(test_loader)

25

### The model


In [1]:
%load_ext autoreload
%autoreload 2
from src import Net 

In [2]:
import torch
model = Net()
print(model)

# Test forward pass
test_input = torch.randn(1, 1, 64, 64)  # Simulated random input
output = model(test_input)
print(output.shape)

Output size after all conv and pool layers: 27040
Net(
  (conv1): Conv2d(1, 60, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(60, 160, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=27040, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
torch.Size([1, 10])


### Train the model

In [ ]:
### reduce the train set to few samples for faster training
### only for tests/ reduces the efficiency of the model
from torch.utils.data import DataLoader, Subset


subset_dataset = Subset(train_dataset, list(range(20)))

train_loader = DataLoader(subset_dataset, batch_size=1, shuffle=False)

In [15]:
len(train_loader)

120

In [16]:
# AVOID YOUR COMPUTER TO CRASH
import torch

torch.set_num_threads(4) 

In [17]:
import multiprocessing


print("Nombre de cœurs physiques:", multiprocessing.cpu_count())


logical_cores = torch.get_num_threads()
print("Nombre de cœurs logiques (threads disponibles):", logical_cores)


Nombre de cœurs physiques: 8
Nombre de cœurs logiques (threads disponibles): 4


In [8]:
import torch
from torch import nn, optim
from src import Net

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = Net().to(device)  
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)  

num_epochs = 80

for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}, start training...")
    model.train()  
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)  

        optimizer.zero_grad()

        outputs = model(inputs)

        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()  

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}")

torch.save(model.state_dict(), "simple_cnn_model_reduced.pth")
print("Model saved successfully.")


Output size after all conv and pool layers: 27040
Epoch 1/80, start training...


d:\WUT\ML\MLProject\src\pytorch_datasets.py:24: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  spectrogram_tensor = torch.load(spectrogram_path)


Epoch [1/80], Loss: 0.8677
Epoch 2/80, start training...
Epoch [2/80], Loss: 0.5781
Epoch 3/80, start training...
Epoch [3/80], Loss: 0.4811
Epoch 4/80, start training...
Epoch [4/80], Loss: 0.4449
Epoch 5/80, start training...
Epoch [5/80], Loss: 0.4140
Epoch 6/80, start training...
Epoch [6/80], Loss: 0.4317
Epoch 7/80, start training...
Epoch [7/80], Loss: 0.3913
Epoch 8/80, start training...
Epoch [8/80], Loss: 0.4065
Epoch 9/80, start training...
Epoch [9/80], Loss: 0.4079
Epoch 10/80, start training...
Epoch [10/80], Loss: 0.3818
Epoch 11/80, start training...
Epoch [11/80], Loss: 0.3798
Epoch 12/80, start training...
Epoch [12/80], Loss: 0.3685
Epoch 13/80, start training...
Epoch [13/80], Loss: 0.3584
Epoch 14/80, start training...
Epoch [14/80], Loss: 0.3545
Epoch 15/80, start training...
Epoch [15/80], Loss: 0.3729
Epoch 16/80, start training...
Epoch [16/80], Loss: 0.3510
Epoch 17/80, start training...
Epoch [17/80], Loss: 0.3566
Epoch 18/80, start training...
Epoch [18/80],

### Evaluate Model

In [9]:
import torch
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

from src import Net
model = Net()

model.load_state_dict(torch.load("simple_cnn_model_reduced.pth"))
print("the model is loaded")
model.eval()

all_labels = []
all_predictions = []

occurence = 0
with torch.no_grad():
    length = len(test_loader)
    for inputs, labels in test_loader:
        print(f"start testing occurence {occurence + 1}/{length}")
        occurence += 1
        inputs, labels = inputs.to(device), labels.to(device)

        
        outputs = model(inputs)
        _, predicted = torch.max(
            outputs, 1
        )  

        
        all_labels.extend(labels.cpu().numpy())  
        all_predictions.extend(predicted.cpu().numpy())

f1 = f1_score(all_labels, all_predictions, average="binary")
precision = precision_score(all_labels, all_predictions, average="binary")
recall = recall_score(all_labels, all_predictions, average="binary")
accuracy = accuracy_score(all_labels, all_predictions)

# Print metrics
print(f"F1-Score: {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Accuracy: {accuracy:.4f}")

Output size after all conv and pool layers: 27040
the model is loaded


C:\Users\elenm\AppData\Local\Temp\ipykernel_22220\506085010.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("simple_cnn_model_reduced.pt

start testing occurence 1/19
start testing occurence 2/19
start testing occurence 3/19
start testing occurence 4/19
start testing occurence 5/19
start testing occurence 6/19
start testing occurence 7/19
start testing occurence 8/19
start testing occurence 9/19
start testing occurence 10/19
start testing occurence 11/19
start testing occurence 12/19
start testing occurence 13/19
start testing occurence 14/19
start testing occurence 15/19
start testing occurence 16/19
start testing occurence 17/19
start testing occurence 18/19
start testing occurence 19/19
F1-Score: 0.9029
Precision: 0.9294
Recall: 0.8778
Accuracy: 0.9433
